# Add LAMMPS Potentials

This Notebook provides details and demonstrations about how PotentialLAMMPS objects can be easily generated for potentials not currently in the database.  This allows for users to easily integrate their personal LAMMPS potentials with the database.

In [1]:
from pathlib import Path
import potentials

## 1. Defining new LAMMPS potentials

For the majority of LAMMPS pair_styles, the build_lammps_potential function can be used to easily generate a PotentialLAMMPS object. The function is a wrapper around numerous subclasses of potentials.build.PotentialLAMMPSBuilder, each of which is designed to properly generate the pair_coeff lines for a given set of pair_styles. There are hundreds of LAMMPS pair_styles, and each pair_style is expecting pair_coeff lines specific to that style. Luckily, the majority of pair_styles have pair_coeff lines that fall into one of two basic formats, and others are known slight variations of that. Thus, a handful of classes can properly capture the vast majority of LAMMPS pair_styles.

__General build_lammps_potential() parameters associated with generating LAMMPS inputs__

- __pair_style__ (*str*) The LAMMPS pair_style option to use.

- __pair_style_terms__ (*list, optional*) Any other terms that appear on the pair_style line (like cutoff) if needed.

- __symbols__ (*list, optional*) The symbols used to identify each unique particle model. Optional if elements is given and the particle symbols are the same as the elemental symbols.

- __elements__ (*list, optional*) The elemental symbols associated with each particle model if the particles represent atoms.

- __masses__ (*list, optional*) The masses of each particle.  Optional if elements is given as standard values can be used.

- __charges__ (*list, optional*) The static charges to assign to each particle, if the model calls for it.

- __allsymbols__ (*bool, optional*) Flag indicating if the coefficient lines must be defined for every particle model in the potential even if those particles are not used.  Default value is False as most pair_styles do not require this.

- __command_terms__ (*list, optional*) Allows any other LAMMPS command lines that must be set for the potential to work properly to be set.  Each command line should be given as a list of terms, and multiple command lines given as a list of lists.

- __units__ (*str, optional*) The LAMMPS units setting to use with the potential, if specific units are required. 

- __atom_style__ (*str, optional*) The LAMMPS atom_style setting to use, if a specific atom_style is required.

Any of the general parameters and pair_style-specific parameters can be directly set as object attributes any time after the object has been created.

### 1.1 Pair potentials

This is the most common format. It is used by all potentials that do not require reading parameter files.

    pair_coeff 1 1 <a_11> <b_11> ...
    pair_coeff 1 2 <a_12> <b_12> ...
    ...

__Format-specific build_lammps_potential() parameters__

- __interactions__ (*list or dict*) Specifies the *symbols* and *terms* associated with each unique pair_coeff interaction.

The associated class also has the set_interaction() method that makes defining the interactions slightly more intuitive.

In [2]:
# Define pair potential with elements Al and Cu
b = potentials.build_lammps_potential(pair_style='lj/cut', elements=['Al', 'Cu'])

# Specify coefficients for each unique interaction (Note: values are junk NOT REAL POTENTIALS!)
b.set_interaction(symbols=['Al', 'Al'], terms=[1.23, 3.412])
b.set_interaction(symbols=['Cu', 'Al'], terms=[1.124, 2.124])
b.set_interaction(symbols=['Cu', 'Cu'], terms=[5.324, 3.14])

# Have the builder return a PotentialLAMMPS object
pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385
mass 2 63.546

pair_style lj/cut
pair_coeff 1 1 1.23 3.412
pair_coeff 1 2 1.124 2.124
pair_coeff 2 2 5.324 3.14



### 1.2 Parameter file potentials

This is the second most common format.  It is used by nearly all potentials that read in a single parameter file and associate atom types to model symbols.

    pair_coeff * * <paramfile> <list of model symbols>

__Format-specific build_lammps_potential() parameters__

- __paramfile__ (*str*) The name of the potential parameter file.

In [3]:
# Define potential for Al.eam.alloy
b = potentials.build_lammps_potential(pair_style='eam/alloy', elements='Al', paramfile='Al.eam.alloy')

# Have the builder return a PotentialLAMMPS object
pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385

pair_style eam/alloy
pair_coeff * * Al.eam.alloy Al



### 1.3 Original EAM pair_style potentials

The original eam pair_style uses a unique format due to it defining elemental models in parameter files and using a universal mixing function.  

    pair_coeff 1 1 <paramfiles[0]>
    pair_coeff 2 2 <paramfiles[1]>
    ...

__Format-specific build_lammps_potential() parameters__

- __paramfiles__ (*list*) The names of the elemental potential parameter files.

In [4]:
# Define potential using the Al.eam and Cu.eam parameter files
b = potentials.build_lammps_potential(pair_style='eam', elements=['Al', 'Cu'], paramfiles=['Al.eam', 'Cu.eam'])

# Have the builder return a PotentialLAMMPS object
pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385
mass 2 63.546

pair_style eam
pair_coeff 1 1 Al.eam
pair_coeff 2 2 Cu.eam



### 1.4 MEAM potentials

MEAM potentials use two types of parameter files: a general library file and a model-specific parameter file.

    pair_coeff * * <libfile> <list of libfile symbols> <paramfile> <list of model symbols>

__Format-specific build_lammps_potential() parameters__

- __libfile__ (*str*) The name of the potential library file.

- __paramfile__ (*str, optional*) The name of the potential parameter file. Might not be used by older MEAM potentials.

*Note*: The MEAM parameter files for alloy (multi-element) models need to know which particle models to associate their parameters with.  As such, the first list of symbols in the pair_coeff line must be in a specific order.  Here, the builder will use the order of symbols/elements parameters as given.

In [5]:
# Define MEAM potential using the lib.meam and Al.meam parameter file
b = potentials.build_lammps_potential(pair_style='meam', elements='Al', libfile='lib.meam', paramfile='Al.meam')

pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385

pair_style meam
pair_coeff * * lib.meam Al Al.meam Al



In [6]:
# Define MEAM potential using the lib.meam only
b = potentials.build_lammps_potential(pair_style='meam', elements='Al', libfile='lib.meam')

# Have the builder return a PotentialLAMMPS object
pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385

pair_style meam
pair_coeff * * lib.meam Al NULL Al



### 1.5 EIM potentials

The eim pair_style uses a slight variation on the parameter file and MEAM formats

    pair_coeff * * <list of libfile symbols> <parameter_file> <list of model symbols>

__Format-specific build_lammps_potential() parameters__

- __paramfile__ (*str*) The name of the potential parameter file.

In [7]:
b = potentials.build_lammps_potential(pair_style='eim', elements=['Al', 'Cu'], paramfile='AlCu.eim')

pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385
mass 2 63.546

pair_style eim
pair_coeff * * Al Cu AlCu.eim Al Cu



### 1.6 KIM potentials

This is used with the older pair_style kim not the newer kim LAMMPS commands.

    pair_coeff * * <list of model symbols>

__Format-specific build_lammps_potential() parameters__

- __kimid__ (*str*) The unique kim ID for the potential.  Note that this is used rather than pair_style_terms.

In [8]:
b = potentials.build_lammps_potential(pair_style='kim', elements='Al', kimid='AKSDFO!')

pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385

pair_style kim AKSDFO!
pair_coeff * * Al



### 1.7 Other potentials

The current builders capture ~95% of all LAMMPS pair_styles. For the remaining pair_styles

- If the pair_style's pair_coeffs follow one of the above formats but is not currently known to follow that format, the underlying builder can be called directly.

- If the pair_style's pair_coeffs do not follow any of the above formats, a new builder can be made and/or the underlying potential_LAMMPS data model can be built by hand. Contact [potentials@nist.gov](mailto:potentials@nist.gov) for help.

## 2. Saving user-defined potentials

The above section descriptions provide a simple means of generating PotentialLAMMPS objects for new potentials not in the system.  This section shows how the same builders can be used to create the underlying potential_LAMMPS data models that can be saved for use later.

__Note__: At the very least id must be set in order to save as id is used as the file name.

### 2.1 Additional metadata

The builders also take additional parameters associated with metadata fields for identifying and classifying the LAMMPS potentials

- __id__ (*str, optional*) A human-readable identifier to name the LAMMPS potential implementation.  Must be set in order to save to the database as the id is used as the potential's file name.

- __key__ (*str, optional*) A UUID4 code to uniquely identify the LAMMPS potential implementation.  If not specified, a new UUID4 code is automatically generated.

- __potid__ (*str, optional*) A human-readable identifier to refer to the conceptual potential model that the potential is based on.  This should be shared by alternate implementations of the same potential.

- __potkey__ (*str, optional*) A UUID4 code to uniquely identify the conceptual potential model. This should be shared by alternate implementations of the same potential. If not specified, a new UUID4 code is automatically generated.

- __status__ (*str, optional*) Indicates if the implementation is 'active' (valid and current), 'superceded' (valid, but better ones exist), or 'retracted' (invalid). Default value is 'active'.

In [9]:
# Start with the eam/alloy example again
pot = potentials.build_lammps_potential(pair_style='eam/alloy', elements='Al', paramfile='Al.eam.alloy')

# Show the generated data model
print(pot.build().json(indent=4))

{
    "potential-LAMMPS": {
        "key": "543154b0-a322-4be5-92fd-49750db56f86", 
        "id": null, 
        "potential": {
            "key": "7dd21151-8030-48a4-a4bc-befa0c4c7439", 
            "id": null
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": {
            "element": "Al"
        }, 
        "pair_style": {
            "type": "eam/alloy"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Al.eam.alloy"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}


Note that the two id fields have null values.  They can be set either during initialization or as attributes after initialization.

In [10]:
pot.id = 'Al-parameterization-round12-v2'
pot.potid = 'Al-parameterization-round12'
# Show the generated data model
print(pot.build().json(indent=4))

{
    "potential-LAMMPS": {
        "key": "543154b0-a322-4be5-92fd-49750db56f86", 
        "id": "Al-parameterization-round12-v2", 
        "potential": {
            "key": "7dd21151-8030-48a4-a4bc-befa0c4c7439", 
            "id": "Al-parameterization-round12"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": {
            "element": "Al"
        }, 
        "pair_style": {
            "type": "eam/alloy"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Al.eam.alloy"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}


### 2.2 Save locally

Any user-defined records for LAMMPS potentials can be saved locally and then later retrieved using potentials.Database.

In [11]:
# Create a dummy parameter file "Al.eam.alloy" 
dummyfile = Path('Al.eam.alloy')

with open(dummyfile, 'w') as f:
    f.write('Dummy parameter file Al.eam.alloy\n')
    f.write('This is created here to demonstrate how to associate parameter files with saved LAMMPS potentials\n')
    f.write('Give paths to already existing parameter files for real potentials\n')
    f.write('This is where parameters would appear if this was a real potential...')

#### 2.2.1 Database.save_lammps_potentials()

Saves a new LAMMPS potential to the local copy of the database

- __lammps_potentials__ (*PotentialLAMMPS or list of PotentialLAMMPS*) The lammps_potential(s) to save.
- __filenames__ (*list, optional*) The path names to any parameter files associated with each lammps_potentials.  Length of the list should be the same as the length of lammps_potentials.  Each entry can be None, a path, or a list of paths.  If the value is None for an entry then the corresponding Potential record will be searched for parameter files to download.   Note that files will only be copied/downloaded if the associated record is new/different.
- __format__ (*str, optional*) The file format to save the record files as.  Allowed values are 'xml' (default) and 'json'.
- __localpath__ (*path-like object, optional*) Path to a local directory where the record and files will be saved to. If not given, will use the localpath value set during object initialization.
- __indent__ (*int, optional*) The indentation spacing size to use for the locally saved record files. If not given, the JSON/XML content will be compact.
- __overwrite__ (*bool, optional*) If True (default) then any matching LAMMPS potentials already in the localpath will be updated if the content has changed.  If False, all existing LAMMPS potentials in the localpath will be unchanged.
- __verbose__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.
- __reload__ (*bool, optional*) If True, will call load_lammps_potentials() after adding the new potential.  Default value is False: loading is slow and should only be done when you wish to retrieve the saved potentials.


In [12]:
# Initialize a Database with localpath testlibrary
potdb = potentials.Database(local=True, remote=False)

# Save to local
potdb.save_lammps_potentials(pot, filenames='Al.eam.alloy', verbose=True, format='json')

1 LAMMPS potentials saved to localpath
 - 1 new potentials added
 - 1 potentials had files copied


In [13]:
# Delete the dummy parameter file
dummyfile.unlink()

#### 2.2.2 load and get

Any local potentials in the Database's localpath can then be found after loading the lammps potentials into memory.

In [14]:
# Load lammps_potentials from the local directory only
potdb.load_lammps_potentials(local=True, remote=False, verbose=True)

Loaded 280 local LAMMPS potentials


In [15]:
# get_lammps_potential will search through the loaded potentials
pot = potdb.get_lammps_potential(id='Al-parameterization-round12-v2')
print(pot)

Al-parameterization-round12-v2


In [16]:
print(pot.pair_info())

mass 1 26.9815385

pair_style eam/alloy
pair_coeff * * C:\Users\lmh1\Documents\library\potential_LAMMPS\Al-parameterization-round12-v2\Al.eam.alloy Al



#### 2.2.3. Delete records

Any LAMMPS potentials saved can also be deleted with the delete_lammps_potential() method.

Parameters
    
- __lammps_potential__ (*PotentialLAMMPS or str*) The LAMMPS potential to delete from the remote database.  Can either give the corresponding PotentialLAMMPS object or just the potential's id/title.
- __local__ (*bool, optional*) If True (default) then the record will be deleted from the localpath.
- __remote__ (*bool, optional*) If True then the record will be deleted from the remote database.   Requires write permissions to potentials.nist.gov.  Default value is False.
- __localpath__ (*path-like object, optional*) Path to a local directory where the file to delete is located.  If not given, will use the localpath value set during object initialization.
- __paramfiles__ (*bool, optional*) If True (default) and local is True, then any parameter files associated with the record will also be deleted from localpath.
- __verbose__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.

In [17]:
potdb.delete_lammps_potential(pot, local=True, remote=False, paramfiles=True, verbose=True)

deleted C:\Users\lmh1\Documents\library\potential_LAMMPS\Al-parameterization-round12-v2.json
